# Análisis Exploratorio de Datos - Clientes Bancarios
# Este notebook realiza el análisis exploratorio de datos de clientes bancarios

In [3]:
# Importar librerías necesarias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('default')
sns.set_theme()
%matplotlib inline

# Importar funciones de carga de datos
import sys
sys.path.append('../src')
from data.data_loader import load_customer_data, load_bank_data, get_basic_info

In [4]:
# Cargar datos de clientes
df_customers = load_customer_data()

# Cargar datos bancarios
df_bank = load_bank_data()

In [5]:
# Información básica de los datos de clientes
print("=== Información de Datos de Clientes ===")
get_basic_info(df_customers)

# Información básica de los datos bancarios
print("\n=== Información de Datos Bancarios ===")
get_basic_info(df_bank)

=== Información de Datos de Clientes ===

Información del DataFrame:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20115 entries, 0 to 20114
Data columns (total 7 columns):
 #   Column             Non-Null Count  Dtype         
---  ------             --------------  -----         
 0   Unnamed: 0         20115 non-null  int64         
 1   Income             20115 non-null  int64         
 2   Kidhome            20115 non-null  int64         
 3   Teenhome           20115 non-null  int64         
 4   Dt_Customer        20115 non-null  datetime64[ns]
 5   NumWebVisitsMonth  20115 non-null  int64         
 6   ID                 20115 non-null  object        
dtypes: datetime64[ns](1), int64(5), object(1)
memory usage: 1.1+ MB
None

Primeras 5 filas:
   Unnamed: 0  Income  Kidhome  Teenhome Dt_Customer  NumWebVisitsMonth  \
0           0  161770        1         0  2012-04-04                 29   
1           1   85477        1         1  2012-12-30                  7   
2         

In [6]:
# Mostrar columnas de cada dataset
print("Columnas en datos de clientes:")
print(df_customers.columns.tolist())

print("\nColumnas en datos bancarios:")
print(df_bank.columns.tolist())

Columnas en datos de clientes:
['Unnamed: 0', 'Income', 'Kidhome', 'Teenhome', 'Dt_Customer', 'NumWebVisitsMonth', 'ID']

Columnas en datos bancarios:
[',age,job,marital,education,default,housing,loan,contact,duration,campaign,pdays,previous,poutcome,emp.var.rate,cons.price.idx,cons.conf.idx,euribor3m,nr.employed,y,date,latitude,longitude,id_']


In [7]:
## 4. Análisis de Valores Nulos

# Undécima celda (code)
# Análisis de valores nulos en datos de clientes
print("=== Valores Nulos en Datos de Clientes ===")
null_customers = df_customers.isnull().sum()
print(null_customers[null_customers > 0])

# Análisis de valores nulos en datos bancarios
print("\n=== Valores Nulos en Datos Bancarios ===")
null_bank = df_bank.isnull().sum()
print(null_bank[null_bank > 0])

=== Valores Nulos en Datos de Clientes ===
Series([], dtype: int64)

=== Valores Nulos en Datos Bancarios ===
Series([], dtype: int64)
